<div align=center>
<img src=https://github.com/datoujinggzj/WhaleDataScienceProject/blob/master/pic/project_logo.jpg?raw=true width='800' />
</div>

今天分享关于【缺失值处理】的基本方法汇总

```diff
- 大家好，我是知识渊博、爱好小酌、嗓音独特，还挺幽默的鲸鲸！
```

今天分享【Data Preparation】的重要环节：缺失值处理。

那么在诊断数据缺失值时，一定要搞清楚为什么会存在缺失值，这将极大影响我们接下来如何处理缺失值！

先介绍一下「缺失类型」

1. MCAR (Missing completely at Random)
2. MAR (Missing at Random)
3. MNAR (Missing not at Random)

MCAR 的意思是“缺失与任何值之间没有关系”。因此，在这种情况下，可以删除缺失值。为此，可以删除列缺失值；最小化丢失数据或删除行缺失值。

MAR 的意思是“缺失与其他观察到的数据之间存在系统关系，但与缺失数据无关”。

最后，MNAR 的意思是“缺失与它的值之间存在关系，缺失或非缺失”。

---


那么如果你是小白的话，我强烈建议花30min阅读本文，本文介绍了常用的4个方法去处理缺失值。


（点击下面跳转）

1.  [None/NaN 的区别](#None和NaN的区别)
2.  [缺失值分类](#缺失值分类)
3.  [缺失值查看](#缺失值查看)
3.  [缺失值移除](#缺失值移除)
4.  [缺失值填充](#缺失值移除)
---
补充：[对缺失值鲁棒性高的模型汇总](#对缺失值鲁棒性高的模型汇总)

# None和NaN的区别

先说说 None/NaN 的区别

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Pandas automatically converts the None to a NaN value. 
# (Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included)
df = pd.DataFrame({'A':[1,2,None], 
                  'B':[5,np.nan,np.nan],
                  'C':[1,2,3]})

In [3]:
None_arr = np.array([1, None, 3, 4])
None_arr

array([1, None, 3, 4], dtype=object)

In [4]:
NaN_arr = np.array([1, np.NaN, 3, 4])
NaN_arr.dtype

dtype('float64')

https://stackoverflow.com/questions/17534106/what-is-the-difference-between-nan-and-none

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |


# 缺失值分类

Rubin在1976年把缺失值归为3类：
1. Missing Completely at Random（MCAR）
2. Missing at Random（MAR）
3. Missing NOT at Random（MNAR）

1. Missing Completely at Random（MCAR）

MCAR是指缺失值产生的原因完全随机，我们无法通过其他已知数据预测该缺失值。

2. Missing at Random（MAR）

MAR指的是缺失值可以用其他的列解释并预测。比如：你求婚失败了，你不知道为啥，那么你可能要关注一下其他的因子（变量），“她是否爱上别人了？”或者“老丈母娘不喜欢你”等等。

3. Missing NOT at Random（MNAR）

MNAR表示缺失值的产生有所原因，比如：一个人很胖，所以他不愿意提供自己的体重。

# 缺失值查看

介绍检查缺失值的简易方法：

- isnull()
- notnull()

In [5]:
df.isnull()

,A,B,C
0,False,False,False
1,False,True,False
2,True,True,False


In [6]:
df.notnull()

,A,B,C
0,True,True,True
1,True,False,True
2,False,False,True


In [7]:
round(df.isnull().sum()/df.shape[0],2)

A    0.33
B    0.67
C    0.00
dtype: float64

In [8]:
df.isnull().sum()/df.shape[0]

A    0.333333
B    0.666667
C    0.000000
dtype: float64

In [9]:
df.isnull().sum(axis=1)/df.shape[1]

0    0.000000
1    0.333333
2    0.666667
dtype: float64

# 缺失值移除

一般缺失值占比在50%左右即可drop

In [10]:
df1 = df.copy()

In [11]:
df1.dropna() # 默认drop axis是行

,A,B,C
0,1.0,5.0,1


In [12]:
df1.dropna(axis=1)

,C
0,1
1,2
2,3


In [13]:
df1.dropna(axis='columns') # 跟上面一样

,C
0,1
1,2
2,3


In [14]:
df1  # inplace 没有设置成 True，所以对 df1 没有原地操作，不影响 df1 本身！

,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2
2,NaN,NaN,3


In [15]:
df1.dropna(axis=0)

,A,B,C
0,1.0,5.0,1


In [16]:
df.dropna(thresh=2)

,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2


# 缺失值填充

介绍几个填充缺失值的常用方法：

#### univariate 单变量
- [fillna](#fillna)
- [replace](#replace)
- [SimpleImputer](#SimpleImputer)

#### multivariate 多变量

- [Impyute](#Impyute)

In [17]:
df2 = df.copy()

### fillna

In [18]:
# 固定值填充
df2.fillna(value='填充') 

,A,B,C
0,1.0,5.0,1
1,2.0,填充,2
2,填充,填充,3


In [19]:
# method='ffill'
df2.fillna(method = 'ffill') # 根据前一个值填充，如果第一个值就是nan，那此方法不适用。

,A,B,C
0,1.0,5.0,1
1,2.0,5.0,2
2,2.0,5.0,3


In [20]:
# method='bfill'
df2.fillna(method = 'bfill') # 根据前一个值填充，如果最后的就是nan，那此方法不适用。

,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2
2,NaN,NaN,3


### replace

In [21]:
# replace
df2.replace(np.nan, -1)

,A,B,C
0,1.0,5.0,1
1,2.0,-1.0,2
2,-1.0,-1.0,3


### SimpleImputer

In [22]:
# SimpleImputer

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer( strategy='mean') #for median imputation replace 'mean' with 'median'
imp_mean.fit(df)
imputed_df = imp_mean.transform(df)  # return array

In [23]:
imputed_df

array([[1. , 5. , 1. ],
       [2. , 5. , 2. ],
       [1.5, 5. , 3. ]])

### Impyute

In [24]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from math import sqrt
import random
import numpy as np
random.seed(0)

#Fetching the dataset
import pandas as pd
dataset = fetch_california_housing()
train, target = pd.DataFrame(dataset.data), pd.DataFrame(dataset.target)
train.columns = ['0','1','2','3','4','5','6','7']
train.insert(loc=len(train.columns), column='target', value=target)

#Randomly replace 40% of the first column with NaN values
column = train['0']
print(column.size)
missing_pct = int(column.size * 0.4)
i = [random.choice(range(column.shape[0])) for _ in range(missing_pct)]
column[i] = np.NaN

20640


In [25]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS

# start the KNN training
imputed_training_impyute=fast_knn(train.values, k=30)

In [26]:
imputed_training_impyute

array([[   3.30117882,   41.        ,    6.98412698, ...,   37.88      ,
        -122.23      ,    4.526     ],
       [   8.3014    ,   21.        ,    6.23813708, ...,   37.86      ,
        -122.22      ,    3.585     ],
       [   4.31326938,   52.        ,    8.28813559, ...,   37.85      ,
        -122.24      ,    3.521     ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,   39.43      ,
        -121.22      ,    0.923     ],
       [   3.40189346,   18.        ,    5.32951289, ...,   39.43      ,
        -121.32      ,    0.847     ],
       [   2.3886    ,   16.        ,    5.25471698, ...,   39.37      ,
        -121.24      ,    0.894     ]])

In [27]:
from impyute.imputation.cs import mice
imputed_training_MICE=mice(train.values)

In [28]:
imputed_training_MICE

array([[   7.04291115,   41.        ,    6.98412698, ...,   37.88      ,
        -122.23      ,    4.526     ],
       [   8.3014    ,   21.        ,    6.23813708, ...,   37.86      ,
        -122.22      ,    3.585     ],
       [   6.93190085,   52.        ,    8.28813559, ...,   37.85      ,
        -122.24      ,    3.521     ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,   39.43      ,
        -121.22      ,    0.923     ],
       [   2.39225991,   18.        ,    5.32951289, ...,   39.43      ,
        -121.32      ,    0.847     ],
       [   2.3886    ,   16.        ,    5.25471698, ...,   39.37      ,
        -121.24      ,    0.894     ]])

In [29]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=30)
imputer.fit(train)
# transform the dataset
imputed_training_KNNImputer = imputer.transform(train)

In [30]:
imputed_training_KNNImputer

array([[   3.11527667,   41.        ,    6.98412698, ...,   37.88      ,
        -122.23      ,    4.526     ],
       [   8.3014    ,   21.        ,    6.23813708, ...,   37.86      ,
        -122.22      ,    3.585     ],
       [   4.33347333,   52.        ,    8.28813559, ...,   37.85      ,
        -122.24      ,    3.521     ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,   39.43      ,
        -121.22      ,    0.923     ],
       [   3.64121333,   18.        ,    5.32951289, ...,   39.43      ,
        -121.32      ,    0.847     ],
       [   2.3886    ,   16.        ,    5.25471698, ...,   39.37      ,
        -121.24      ,    0.894     ]])

## 对缺失值鲁棒性高的模型汇总

![](../../../pic/missing_value_algo.png)

# Great Job!